Module 01: Exploratory Data Analysis for Demand & Inventory

This notebook performs exploratory data analysis (EDA) for Module 01 of the **"Intelligent System for Supply Chain Management"** project.  

The primary goal is to optimize inventory and purchasing management, with a target of **reducing overstocking by 20%** within six months.

---

## Data Acquisition
### Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import json
import plotly.express as px
import plotly.io as pio

from plotly.subplots import make_subplots
from sklearn.preprocessing import OrdinalEncoder
from smart_supply_chain_ai.data_processing import get_data

import warnings
warnings.filterwarnings('ignore')

# Set up display options and plotting template
pd.set_option('display.max_columns', None)
pio.templates.default = "plotly_white"
px.defaults.width = 800
px.defaults.height = 600

### Load Dataset

In [2]:
# Define data paths
raw_data_path = os.path.join('../data', 'raw')

In [3]:
# Download Data from Kaggle
# link for data on web - [USER] / [DATASET_NAME]
module_one = "salahuddinahmedshuvo/grocery-inventory-and-sales-dataset"
# Download Data and Unzip 
get_data.download_kaggle_dataset(module_one, raw_data_path);

Starting the download of dataset 'salahuddinahmedshuvo/grocery-inventory-and-sales-dataset' from Kaggle...
Dataset URL: https://www.kaggle.com/datasets/salahuddinahmedshuvo/grocery-inventory-and-sales-dataset
Download, unzipping, and cleanup complete! The dataset was saved to: ../data/raw


In [4]:
# Load the raw dataset
df_raw = pd.read_csv(raw_data_path + '/Grocery_Inventory_and_Sales_Dataset.csv')

## Data Cleaning and Preprocessing

In [5]:
# Description of dataset columns
column_inventory = {
    'Product_ID': 'Unique identifier for each product.',
    'Product_Name': 'Name of the product.',
    'Category': 'The product category (e.g., Grains & Pulses, Beverages, Fruits & Vegetables).',
    'Supplier_ID': 'Unique identifier for the product supplier.',
    'Supplier_Name': 'Name of the supplier.',
    'Stock_Quantity': 'The current stock level of the product in the warehouse.',
    'Reorder_Level': 'The stock level at which new stock should be ordered.',
    'Reorder_Quantity': 'The quantity of product to order when the stock reaches the reorder level.',
    'Unit_Price': 'Price per unit of the product.',
    'Date_Received': 'The date the product was received into the warehouse.',
    'Last_Order_Date': 'The last date the product was ordered.',
    'Expiration_Date': 'The expiration date of the product, if applicable.',
    'Warehouse_Location': 'The warehouse address where the product is stored.',
    'Sales_Volume': 'The total number of units sold.',
    'Inventory_Turnover_Rate': 'The rate at which the product sells and is replenished.',
    'Status': 'Current status of the product (e.g., Active, Discontinued, Backordered).',
    'Stock_Value': 'The total monetary value of the current stock (Stock_Quantity * Unit_Price).',
    'Days_For_Expiration': 'The number of days until the product expires. Negative values indicate the product is already expired.',
    'Expiration_Status': 'Categorical status based on the expiration date (e.g., Expired, Nearing, Safe).',
    'Purchase_Order': 'The total monetary value of the new order (Reorder_Quantity * Unit_Price), used to analyze discrepancies.',
    'DOI_Inventory_Turnover': 'Days of Inventory. The number of days the current inventory can last, based on the annual sales rate.',
    'Delivery_Lag': 'The number of days between the last order date and the date the product was received.',
    'Delivery_Lag_Issue': 'Flag records with inconsistent delivery lag (negative values).',
}

In [6]:
# Create a copy for cleaning and preprocessing
df = df_raw.copy()

In [7]:
# Standardize Column Headers
# Rename 'Catagory' to 'Category' for consistency
df.rename(columns={"Catagory": "Category"}, inplace=True)

In [8]:
# Verify for missing values in the 'Category' column
df[df['Category'].isna()]

,Product_ID,Product_Name,Category,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Last_Order_Date,Expiration_Date,Warehouse_Location,Sales_Volume,Inventory_Turnover_Rate,Status
685,10-378-9729,Cabbage,NaN,83-941-9620,Rooxo,69,21,68,$66.55,12/23/2024,11/26/2024,9/21/2024,2 Butterfield Pass,36,35,Discontinued


In [9]:
# List unique categories
df.Category.unique()

array(['Grains & Pulses', 'Beverages', 'Fruits & Vegetables',
       'Oils & Fats', 'Dairy', 'Bakery', 'Seafood', nan], dtype=object)

### Handle Missing Data

In [10]:
# The 'Category' column has one missing value, corresponding to 'Cabbage'.  
# Based on domain knowledge, we'll fill this with 'Fruits & Vegetables'
df = df.fillna('Fruits & Vegetables')

### Convert Data Types for Analysis

In [11]:
# Check the data types and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Product_ID               990 non-null    object
 1   Product_Name             990 non-null    object
 2   Category                 990 non-null    object
 3   Supplier_ID              990 non-null    object
 4   Supplier_Name            990 non-null    object
 5   Stock_Quantity           990 non-null    int64 
 6   Reorder_Level            990 non-null    int64 
 7   Reorder_Quantity         990 non-null    int64 
 8   Unit_Price               990 non-null    object
 9   Date_Received            990 non-null    object
 10  Last_Order_Date          990 non-null    object
 11  Expiration_Date          990 non-null    object
 12  Warehouse_Location       990 non-null    object
 13  Sales_Volume             990 non-null    int64 
 14  Inventory_Turnover_Rate  990 non-null    i

In [12]:
# Convert date columns to datetime objects
date_columns = ['Date_Received', 'Last_Order_Date', 'Expiration_Date']
df[date_columns] = df[date_columns].apply(pd.to_datetime, errors='coerce')

In [13]:
# Clean and convert 'Unit_Price' to a numeric format
df['Unit_Price'] = df['Unit_Price'].str.replace('$', '').astype('float')

In [14]:
# Convert categorical columns to the 'category' type for memory efficiency
cat_columns = ['Category', 'Status']
df[cat_columns] = df[cat_columns].astype('category')

In [15]:
# Statistics for Numeric columns
df.describe(exclude=['datetime', 'object', 'category']).T

,count,mean,std,min,25%,50%,75%,max
Stock_Quantity,990.0,55.609091,26.300775,10.0,33.00,56.000,79.00,100.00
Reorder_Level,990.0,51.215152,29.095241,1.0,25.25,53.000,77.00,100.00
Reorder_Quantity,990.0,51.913131,29.521059,1.0,25.00,54.000,77.00,100.00
Unit_Price,990.0,5.924192,6.491280,0.2,2.50,4.225,7.00,98.43
Sales_Volume,990.0,58.925253,23.002318,20.0,39.00,58.000,78.00,100.00
Inventory_Turnover_Rate,990.0,50.150505,28.798954,1.0,25.00,50.000,74.75,100.00


In [16]:
# Statistics for Categorical columns
df.describe(include=['category']).T

,count,unique,top,freq
Category,990,7,Fruits & Vegetables,332
Status,990,3,Discontinued,333


In [17]:
# Statistics for String columns
df.describe(include=['object']).T

,count,unique,top,freq
Product_ID,990,990,29-205-1132,1
Product_Name,990,121,Bread Flour,19
Supplier_ID,990,990,38-037-1699,1
Supplier_Name,990,350,Katz,12
Warehouse_Location,990,990,48 Del Sol Trail,1


In [18]:
# Display the minimum date for each column
df[['Date_Received', 'Last_Order_Date', 'Expiration_Date']].min()

Date_Received     2024-02-25
Last_Order_Date   2024-02-25
Expiration_Date   2024-02-25
dtype: datetime64[ns]

In [19]:
# Display the maximum date for each column
df[['Date_Received', 'Last_Order_Date', 'Expiration_Date']].max()

Date_Received     2025-02-24
Last_Order_Date   2025-02-24
Expiration_Date   2025-02-24
dtype: datetime64[ns]

In [20]:
# Check for duplicate product IDs
df['Product_ID'].duplicated().sum()

0

In [21]:
# Check for duplicate supplier IDs
df['Supplier_ID'].duplicated().sum()

0

# Feature Engineering: Create New Metrics

In [22]:
# Calculate the 'Stock_Value'
df['Stock_Value'] = df['Stock_Quantity'] * df['Unit_Price']

In [23]:
# Calculate the 'Days_For_Expiration'
df['Days_For_Expiration'] = (df['Expiration_Date'] - df['Date_Received']).dt.days.astype('Int64')

In [24]:
# Replace negative numbers for zero
df['Days_For_Expiration'].where(df['Days_For_Expiration'] >= 0, 0, inplace=True)

In [25]:
# Create 'Expiration_Status' (Expired, Nearing, Safe)
df['Expiration_Status'] = np.where(df['Days_For_Expiration'] == 0, 'Expired', 
                                         np.where(df['Days_For_Expiration'] < 30, 'Nearing', 'Safe'))
df['Expiration_Status'] = df['Expiration_Status'].astype('category')

In [26]:
# Calculate 'Stock_Coverage' in days (based on 1-year sales window)
# Use the 'Inventory_Turnover_Rate' to calculate Days of Inventory (DOI)
df['DOI_Inventory_Turnover'] = (365 / df['Inventory_Turnover_Rate']).apply(np.floor).astype('int')

In [27]:
# Create 'Purchase_Order' from 'Reorder_Quantity' to check for discrepancies
df['Purchase_Order'] = df['Reorder_Quantity'] * df['Unit_Price']

In [28]:
# Calculate the supplier delivery lag (number of days between order placement and receipt)
df['Delivery_Lag'] = (df['Date_Received'] - df['Last_Order_Date']).dt.days

### Proposed Metric for Imputing Negative Values in Delivery_Lag
An analysis of the dataset revealed that **more than 50%** of the Delivery_Lag values are negative.  
These entries have been flagged separately and excluded from statistical calculations to avoid distorting the distribution of valid data.

To address the presence of these invalid values, a metric is proposed to impute and correct them.  
Ideally, the most appropriate approach would involve consulting subject-matter experts to determine the correct values for each specific case.  
However, since this project relies solely on publicly available data, such a strategy is not feasible.

As an alternative, synthetic data will be generated. Using the valid Delivery_Lag records, the standard deviation of the distribution will be calculated.  
Based on this parameter, random lag values will be generated that reflect the observed variability in the original data, and these will be used to replace the negative entries.

In [29]:
# Select only rows where Delivery_Lag is negative (less than 0)
df[['Delivery_Lag', 'Status']].query('Delivery_Lag < 0')

,Delivery_Lag,Status
3,-73,Active
4,-100,Backordered
6,-50,Active
11,-53,Backordered
12,-58,Discontinued
...,...,...
983,-55,Active
985,-113,Active
986,-1,Active
987,-21,Active


In [30]:
df['Delivery_Lag'].describe()

count    990.000000
mean      -2.710101
std      150.485792
min     -343.000000
25%     -111.000000
50%       -6.000000
75%      110.000000
max      352.000000
Name: Delivery_Lag, dtype: float64

In [31]:
# Flag records with inconsistent delivery lag (negative values)
df['Delivery_Lag_Issue'] = df['Delivery_Lag'] < 0

# Replace zero or negative delivery lag values with NaN to exclude them from calculations
df['Delivery_Lag'] = df['Delivery_Lag'].where(df['Delivery_Lag'] > 0, np.nan)

In [32]:
# Calculates the percentage of records in the dataset with a negative Delivery_Lag 
# (flagged as Delivery_Lag_Issue = True) relative to the total number of rows.
print(f'Percentage of DataFrame with negative Delivery Lag: {(df.Delivery_Lag_Issue.sum() / df.shape[0]) * 100:.2f} %')

Percentage of DataFrame with negative Delivery Lag: 51.92 %


In [33]:
# Remove missing values from the Delivery_Lag column
valid_values = df['Delivery_Lag'].dropna()

# Create a histogram to visualize the distribution of valid Delivery_Lag values
fig = px.histogram(
    valid_values,
    title='Distribution of Valid Delivery Lag Values (in Days)'
)
fig.update_layout(
    xaxis_title='Delivery Lag (Days)',
    yaxis_title='Frequency',
    bargap=0.1,
    showlegend=False
)

# Display the histogram
fig.show()


In [34]:
n_missing = df['Delivery_Lag'].isna().sum()

rng = np.random.default_rng(42)

synthetic = rng.choice(valid_values, size=n_missing, replace=True)

In [35]:
# Create subplot figure with 2 horizontal panels
fig = make_subplots(rows=1, cols=2, x_title='Delivery Lag (in Days)', y_title='Frequency',
                       subplot_titles=['Distribution of Valid Delivery Lag Values (in Days)', 
                                     'Distribution of Synthetic Delivery Lag Values (in Days)'])

# Create histogram plots to visualize distributions
fig1 = px.histogram(valid_values,)
fig2 = px.histogram(synthetic, color_discrete_sequence=['green'])

# Add plots to subfigure
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)

# Adjust layout and display
fig.update_layout(showlegend=False, bargap=0.1, width=1000)
fig.show()

In [36]:
# Impute missing Delivery_Lag values with synthetic data generated from the valid distribution
df.loc[df['Delivery_Lag'].isna(), 'Delivery_Lag'] = synthetic
# Transform float number in integer
df['Delivery_Lag'] = df.Delivery_Lag.astype(int)


In [37]:
# Plot histogram to assess the distribution of Delivery_Lag values after data cleaning and imputation
fig = make_subplots(rows=1, cols=2, x_title='Delivery Lag (in Days)', y_title='Frequency',
                       subplot_titles=['Distribution of Delivery Lag values after data imputation', 'Distribution of Synthetic Delivery Lag Values (in Days)'])

# Create histogram plots to visualize distributions
fig1 = px.histogram(df.Delivery_Lag, color_discrete_sequence=['purple'])
fig2 = px.histogram(valid_values, color_discrete_sequence=['blue'])

# Add plots to subfigure
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)

# Adjust layout and display
fig.update_layout(showlegend=False, bargap=0.1, height=500, width=1000)
fig.show()


# Exploratory Data Analysis (EDA)

In [38]:
# Descriptive Statistics
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Stock_Quantity,990.0,55.609091,10.0,33.0,56.0,79.0,100.0,26.300775
Reorder_Level,990.0,51.215152,1.0,25.25,53.0,77.0,100.0,29.095241
Reorder_Quantity,990.0,51.913131,1.0,25.0,54.0,77.0,100.0,29.521059
Unit_Price,990.0,5.924192,0.2,2.5,4.225,7.0,98.43,6.49128
Date_Received,990,2024-08-23 02:18:10.909090816,2024-02-25 00:00:00,2024-05-27 00:00:00,2024-08-19 00:00:00,2024-11-23 00:00:00,2025-02-24 00:00:00,NaN
Last_Order_Date,990,2024-08-25 19:20:43.636363520,2024-02-25 00:00:00,2024-05-29 00:00:00,2024-08-20 12:00:00,2024-11-29 00:00:00,2025-02-24 00:00:00,NaN
Expiration_Date,990,2024-08-23 06:45:49.090909184,2024-02-25 00:00:00,2024-05-23 00:00:00,2024-08-23 12:00:00,2024-11-23 00:00:00,2025-02-24 00:00:00,NaN
Sales_Volume,990.0,58.925253,20.0,39.0,58.0,78.0,100.0,23.002318
Inventory_Turnover_Rate,990.0,50.150505,1.0,25.0,50.0,74.75,100.0,28.798954
Stock_Value,990.0,336.014859,2.0,108.0,209.0,413.75,5512.08,435.422617


### Inventory and Product Status Analysis

In [39]:
# Distribution of products by status (Active, Discontinued, Backordered)
df.Status.value_counts()
# Visual: Bar plot of product status

Status
Discontinued    333
Active          332
Backordered     325
Name: count, dtype: int64

In [40]:
fig = px.bar(df, 'Status', title='Product Status')
fig.show()

In [41]:
# Identify and quantify inactive and expired stock
print(f"Total Products Expired: {df[df['Expiration_Status'] == 'Expired'].shape[0]}") # Count expired products
print(f"Total Products Discontinued: {df[df['Status'] == 'Discontinued'].shape[0]}") # Count discontinued products

Total Products Expired: 499
Total Products Discontinued: 333


In [42]:
# Financial value of expired stock
total = df[df['Expiration_Status'] == 'Expired']['Stock_Value'].sum()
print(f'Total Value of expired stock products {total:,}')

Total Value of expired stock products 173,763.53


### Stock Value Analysis by Category and Supplier

In [43]:
# Top categories by total stock value
df.groupby('Category')['Stock_Value'].sum().sort_values(ascending=False)

Category
Fruits & Vegetables    90625.11
Beverages              62942.25
Seafood                62515.90
Dairy                  50601.95
Grains & Pulses        31969.20
Oils & Fats            17211.50
Bakery                 16788.80
Name: Stock_Value, dtype: float64

In [44]:
# Visual: Bar plot of stock value by category
fig = px.histogram(df, x='Category', y='Stock_Value', title='Stock Value by Category')
fig.show()

In [45]:
# Analysis of supplier performance (delivery lag and stock value)
# Suppliers with high stock value
df_supplier_stock = df.groupby(by=['Supplier_Name', 'Stock_Quantity'], as_index=False)['Stock_Value'].sum().sort_values(by='Stock_Value', ascending=False)

In [46]:
# Visual: Scatter plot of stock quantity vs. stock value by supplier
fig = px.scatter(df_supplier_stock, x='Stock_Quantity', y='Stock_Value', color='Supplier_Name',
                 title='Stock Quantity vs. Stock Value by Supplier')
fig.update_layout(height=600, width=900)
fig.show()

### Stock Coverage and Risk Analysis

In [47]:
# Identify products with low stock coverage (less than 8 days)
df[df['DOI_Inventory_Turnover'] < 8].shape[0]

542

In [48]:
# Visual: Histogram of 'DOI_Inventory_Turnover' to show the distribution
fig = px.histogram(df.query('DOI_Inventory_Turnover < 8'), 'DOI_Inventory_Turnover', title='Distribution Stock Coverage in days')
fig.update_layout(bargap=0.1)
fig.show()

In [49]:
# Identify potential discrepancies
# Compare 'Stock_Quantity' with 'Reorder_Level' for active products
print('Active Products with Stock Quantities Inconsistent with Reorder Levels')
print(f"Total: {df[(df['Status'] == 'Active') & (df['Stock_Quantity'] < df['Reorder_Level']) & (df['Stock_Quantity'] < df['Reorder_Level'])].shape[0]}")
print(f"Products Expired: {df[(df['Status'] == 'Active') & (df['Stock_Quantity'] < df['Reorder_Level']) & (df['Expiration_Status'] == 'Expired')].shape[0]}")
print(f"Products Nearing: {df[(df['Status'] == 'Active') & (df['Stock_Quantity'] < df['Reorder_Level']) & (df['Expiration_Status'] == 'Nearing')].shape[0]}")
print(f"Products Safe: {df[(df['Status'] == 'Active') & (df['Stock_Quantity'] < df['Reorder_Level']) & (df['Expiration_Status'] == 'Safe')].shape[0]}")


Active Products with Stock Quantities Inconsistent with Reorder Levels
Total: 136
Products Expired: 68
Products Nearing: 8
Products Safe: 60


In [50]:
# Visual: Scatter plot Stock Quantity vs. Reorder_Level with Expired Status
fig = px.scatter(df.query('Status == "Active" and Stock_Quantity < Reorder_Level'), x='Stock_Quantity', y='Reorder_Level',
           title='Active Products: Stock and Reorder Level Discrepancies', color='Expiration_Status', labels={'Expiration_Status': 'Status'}, )

fig.show()

# Key Insights and Next Steps

### Summary of Key Findings

- A large number of products are inactive or expired, representing significant potential for capital recovery and waste reduction.
- There are notable discrepancies between current stock and reorder levels, indicating a potential mismatch between inventory and purchasing policies.
- Top stock categories by value are Fruits & Vegetables, Seafood, and Dairy.
- A number of products have a very low stock coverage (less than 8 days), presenting a high risk of stockouts.
- Supplier analysis reveals significant delivery delays, which directly impacts inventory planning.

### Recommended Next Steps


Based on this EDA, the next steps for the project include:
1. Developing a demand forecasting model to predict future sales volume.
2. Building an inventory optimization model that considers demand forecasts, supplier lead times, and product expiration dates.
3. Simulating different purchasing scenarios to find the optimal balance between cost, service level, and capital utilization.

In [51]:
# Define data paths
processed_data_path = os.path.join('../data', 'processed')

utils_data_path = os.path.join('../docs/column_descriptions.json')

In [52]:
# Sort DataFrame by Date_Received in ascending order
df = df.sort_values(by='Date_Received').reset_index(drop=True)

In [53]:
# Save Data
df.to_pickle(processed_data_path + '/grocery.pkl')

# save Dictionary JSON archive
with open(utils_data_path, 'w') as f:
    json.dump(column_inventory, f, indent=4)